# Chapter 10: Bayesian Workflow — Diagnostics and Best Practices

We revisit the transfer learning scenario from Chapter 07 (NYC temperature → Citi Bike sales) and augment it with:

1. **Prior predictive checks** — Do the priors produce plausible data *before* fitting?
2. **Convergence diagnostics** — R-hat, ESS, trace plots, energy plots
3. **Posterior predictive checks** — Does the fitted model generate data that looks like the observations?
4. **Model comparison** — WAIC / LOO-CV between baseline and transfer models
5. **Prior sensitivity analysis** — How sensitive are forecasts to prior choices?
6. **Full posterior summaries** — HDI, credible intervals, not just point estimates
7. **Prior-to-posterior visualisation** — Showing how the data updates our beliefs

These steps form the core of the WAMBS (When to Worry and how to Avoid the Misuse of Bayesian Statistics) checklist.

## Setup and Imports

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az

from vangja import LinearTrend, FourierSeasonality, FlatTrend
from vangja.datasets import load_citi_bike_sales, load_nyc_temperature
from vangja.utils import (
    metrics,
    compare_models,
    plot_prior_predictive,
    plot_posterior_predictive,
    plot_prior_posterior,
    prior_sensitivity_analysis,
)

az.style.use("arviz-darkgrid")

np.random.seed(42)

print("Imports successful!")

---

## 1. Data Loading

We reproduce the setup from Chapter 07: a long NYC temperature series (source) and a
short Citi Bike sales series (target with ~3 months training data).

In [ ]:
# Load datasets
temp_df = load_nyc_temperature()
sales_df = load_citi_bike_sales()

# Match date ranges as in the original blog post
temp_df = temp_df[
    (temp_df["ds"] >= sales_df["ds"].min()) & (temp_df["ds"] <= sales_df["ds"].max())
]

# Train/test split for sales
train_test_date = pd.to_datetime("2013-10-15")
sales_train = sales_df[sales_df["ds"] < train_test_date].copy()
sales_test = sales_df[sales_df["ds"] >= train_test_date].copy()

print(
    f"Temperature: {temp_df.shape[0]} days  ({temp_df['ds'].min().date()} – {temp_df['ds'].max().date()})"
)
print(
    f"Sales train: {sales_train.shape[0]} days ({sales_train['ds'].min().date()} – {sales_train['ds'].max().date()})"
)
print(
    f"Sales test:  {sales_test.shape[0]} days ({sales_test['ds'].min().date()} – {sales_test['ds'].max().date()})"
)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 7))

axes[0].plot(temp_df["ds"], temp_df["y"], "C0-", lw=0.5, alpha=0.7)
axes[0].set_title("NYC Daily Max Temperature (source series)")
axes[0].set_ylabel("Temperature (°F)")

axes[1].plot(sales_train["ds"], sales_train["y"], "C0o-", ms=2, lw=0.5, label="Train")
axes[1].plot(sales_test["ds"], sales_test["y"], "C1o-", ms=2, lw=0.5, label="Test")
axes[1].axvline(train_test_date, color="gray", ls="--", lw=2)
axes[1].set_title("Citi Bike Sales (target series)")
axes[1].set_ylabel("Rides")
axes[1].legend()

for ax in axes:
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## 2. Prior Predictive Checks

**Why?** Before fitting, we should verify that our priors produce data in a
plausible range. If the prior predictive distribution generates wildly
unreasonable values, our priors need adjustment. This is the first step in
the WAMBS checklist.

We compare prior predictives from:
- A **baseline** model (default uninformed priors)
- The **temperature** model (simple priors, long data)

### 2.1 Temperature Model — Prior Predictive

In [ ]:
# Define the temperature model
temp_model = FlatTrend() + FourierSeasonality(period=365.25, series_order=6)

# We need to fit first to build the PyMC model graph, then sample from the prior
# Fit with NUTS for full posterior (needed for diagnostics later)
print("Fitting temperature model with NUTS...")
temp_model.fit(temp_df, method="nuts", scaler="minmax")
print("Done!")

In [ ]:
# Sample from the prior predictive
temp_prior_pred = temp_model.sample_prior_predictive(samples=500)

fig, ax = plt.subplots(figsize=(14, 5))
plot_prior_predictive(
    temp_prior_pred,
    data=temp_model.data,
    n_samples=100,
    ax=ax,
    title="Temperature Model — Prior Predictive Check",
)
plt.tight_layout()
plt.show()

print("Each thin blue line is one draw from the prior.")
print("The orange line is the observed temperature data.")
print("Prior predictive should at least *contain* the observed data range.")

### 2.2 Baseline Sales Model — Prior Predictive

The baseline model uses default uninformative priors for yearly seasonality.
With only ~3 months of data, the prior predictive may be very wide.

In [ ]:
# Baseline model (same as Ch07 but fit with NUTS for diagnostics)
baseline_model = (
    LinearTrend(n_changepoints=5)
    + FourierSeasonality(period=365.25, series_order=6)
    + FourierSeasonality(period=91.31, series_order=4)
    + FourierSeasonality(period=30.44, series_order=3)
    + FourierSeasonality(period=7, series_order=2)
)

print("Fitting baseline sales model with NUTS...")
baseline_model.fit(sales_train, method="nuts", scaler="minmax")
print("Done!")

In [ ]:
baseline_prior_pred = baseline_model.sample_prior_predictive(samples=500)

fig, ax = plt.subplots(figsize=(14, 5))
plot_prior_predictive(
    baseline_prior_pred,
    data=baseline_model.data,
    n_samples=100,
    ax=ax,
    title="Baseline Sales Model — Prior Predictive Check",
)
plt.tight_layout()
plt.show()

print("With default priors, the prior predictive is very diffuse.")
print("This is expected for weakly informative priors.")

---

## 3. Convergence Diagnostics

After fitting with MCMC, we **must** verify that the sampler has converged. Key checks:

| Diagnostic | Threshold | Meaning |
|---|---|---|
| R-hat | < 1.01 | Chains have mixed well |
| ESS bulk | > 400 | Enough effective samples for the bulk of the posterior |
| ESS tail | > 400 | Enough effective samples for the tails |
| BFMI | > 0.3 | No energy transition problems (NUTS) |
| Divergences | 0 | No pathological geometry |

### 3.1 Temperature Model Convergence

In [ ]:
# Convergence summary for the temperature model
temp_summary = temp_model.convergence_summary()
display(temp_summary)

# Check thresholds
rhat_ok = (temp_summary["r_hat"] < 1.01).all()
ess_ok = (temp_summary["ess_bulk"] > 400).all() and (
    temp_summary["ess_tail"] > 400
).all()
print(f"\nR-hat < 1.01 for all parameters: {rhat_ok}")
print(f"ESS > 400 for all parameters:     {ess_ok}")

In [ ]:
# Trace plots — visual inspection of mixing
temp_model.plot_trace()
plt.suptitle("Temperature Model — Trace Plots", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Energy plot (BFMI diagnostic for NUTS)
temp_model.plot_energy()
plt.suptitle("Temperature Model — Energy Diagnostic", y=1.02)
plt.tight_layout()
plt.show()

print("The marginal energy and energy transition distributions should overlap well.")
print("Large discrepancy indicates the sampler struggles to explore the posterior.")

### 3.2 Baseline Sales Model Convergence

In [ ]:
baseline_summary = baseline_model.convergence_summary()
display(baseline_summary)

rhat_ok = (baseline_summary["r_hat"] < 1.01).all()
ess_ok = (baseline_summary["ess_bulk"] > 400).all() and (
    baseline_summary["ess_tail"] > 400
).all()
print(f"\nR-hat < 1.01 for all parameters: {rhat_ok}")
print(f"ESS > 400 for all parameters:     {ess_ok}")

In [ ]:
baseline_model.plot_trace()
plt.suptitle("Baseline Sales Model — Trace Plots", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
baseline_model.plot_energy()
plt.suptitle("Baseline Sales Model — Energy Diagnostic", y=1.02)
plt.tight_layout()
plt.show()

---

## 4. Posterior Predictive Checks

**Why?** If the model fits well, data simulated from the posterior should
resemble the observed data. Large discrepancies signal model misspecification.

### 4.1 Temperature Model

In [ ]:
temp_ppc = temp_model.sample_posterior_predictive()

fig, ax = plt.subplots(figsize=(14, 5))
plot_posterior_predictive(
    temp_ppc,
    data=temp_model.data,
    n_samples=100,
    ax=ax,
    title="Temperature Model — Posterior Predictive Check",
)
plt.tight_layout()
plt.show()

print("Blue traces = simulated data from posterior; orange = observed.")
print("The simulated data should bracket the observations.")

### 4.2 Baseline Sales Model

In [ ]:
baseline_ppc = baseline_model.sample_posterior_predictive()

fig, ax = plt.subplots(figsize=(14, 5))
plot_posterior_predictive(
    baseline_ppc,
    data=baseline_model.data,
    n_samples=100,
    ax=ax,
    title="Baseline Sales Model — Posterior Predictive Check",
)
plt.tight_layout()
plt.show()

---

## 5. Transfer Learning Model

Now we build the transfer model: yearly seasonality is transferred from the temperature posterior via `tune_method="parametric"`. We also fit with NUTS so we can run the full diagnostic suite.

In [ ]:
transfer_model = (
    FlatTrend()
    + FourierSeasonality(
        period=365.25,
        series_order=6,
        tune_method="parametric",
    )
    + FourierSeasonality(period=91.31, series_order=4)
    + FourierSeasonality(period=30.44, series_order=3)
    + FourierSeasonality(period=7, series_order=2)
)

print("Fitting transfer-learning sales model with NUTS...")
transfer_model.fit(
    sales_train,
    method="nuts",
    idata=temp_model.trace,
    t_scale_params=temp_model.t_scale_params,
    scaler="minmax",
)
print("Done!")

In [ ]:
# Convergence diagnostics for the transfer model
transfer_summary = transfer_model.convergence_summary()
display(transfer_summary)

rhat_ok = (transfer_summary["r_hat"] < 1.01).all()
ess_ok = (transfer_summary["ess_bulk"] > 400).all() and (
    transfer_summary["ess_tail"] > 400
).all()
print(f"\nR-hat < 1.01 for all parameters: {rhat_ok}")
print(f"ESS > 400 for all parameters:     {ess_ok}")

In [ ]:
transfer_model.plot_trace()
plt.suptitle("Transfer Model — Trace Plots", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
transfer_model.plot_energy()
plt.suptitle("Transfer Model — Energy Diagnostic", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Posterior predictive check for transfer model
transfer_ppc = transfer_model.sample_posterior_predictive()

fig, ax = plt.subplots(figsize=(14, 5))
plot_posterior_predictive(
    transfer_ppc,
    data=transfer_model.data,
    n_samples=100,
    ax=ax,
    title="Transfer Model — Posterior Predictive Check",
)
plt.tight_layout()
plt.show()

---

## 6. Full Posterior Summaries

Instead of reporting single point estimates, we should report **full posterior summaries** including credible intervals (HDI).

In [ ]:
print("=" * 70)
print("TEMPERATURE MODEL — Full Posterior Summary")
print("=" * 70)
display(temp_model.summary())

print("\n" + "=" * 70)
print("TRANSFER MODEL — Full Posterior Summary")
print("=" * 70)
display(transfer_model.summary())

In [ ]:
# Posterior density plots for key parameters
print("Temperature Model — Posterior Densities:")
temp_model.plot_posterior()
plt.suptitle("Temperature Model — Posterior Densities", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
print("Transfer Model — Posterior Densities:")
transfer_model.plot_posterior()
plt.suptitle("Transfer Model — Posterior Densities", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

---

## 7. Model Comparison (WAIC / LOO-CV)

Information criteria allow principled comparison of models without held-out data. LOO-CV (via PSIS) is generally preferred over WAIC as it is more robust.

**Note:** Both models must be fitted with MCMC/VI and the traces must contain log-likelihood values for WAIC/LOO to work. In some pymc versions this is only available when `idata_kwargs={"log_likelihood": True}` is passed. Vangja calculates the log-likelihood if it is not already present in the trace.

In [ ]:
# Individual WAIC / LOO scores
print("Baseline model — LOO:")
try:
    baseline_loo = baseline_model.loo()
    print(baseline_loo)
except Exception as e:
    print(f"LOO not available: {e}")

print("\nTransfer model — LOO:")
try:
    transfer_loo = transfer_model.loo()
    print(transfer_loo)
except Exception as e:
    print(f"LOO not available: {e}")

In [ ]:
# Individual WAIC scores
print("Baseline model — WAIC:")
try:
    baseline_waic = baseline_model.waic()
    print(baseline_waic)
except Exception as e:
    print(f"WAIC not available: {e}")

print("\nTransfer model — WAIC:")
try:
    transfer_waic = transfer_model.waic()
    print(transfer_waic)
except Exception as e:
    print(f"WAIC not available: {e}")

---

## 8. Prior Sensitivity Analysis

A central recommendation of the WAMBS checklist: test whether your conclusions
change when you vary the priors. If results are highly sensitive to the prior,
you need more data or a better-justified prior.

We vary `beta_sd` (the standard deviation of the Fourier coefficient prior) on
the **yearly seasonality** in the baseline model. A small `beta_sd` constrains
the seasonal amplitude; a large one allows wild oscillations.

In [ ]:
def make_baseline(beta_sd=10):
    """Factory: baseline sales model with variable yearly-seasonality prior."""
    return (
        LinearTrend(n_changepoints=5)
        + FourierSeasonality(period=365.25, series_order=6, beta_sd=beta_sd)
        + FourierSeasonality(period=91.31, series_order=4)
        + FourierSeasonality(period=30.44, series_order=3)
        + FourierSeasonality(period=7, series_order=2)
    )


print("Running prior sensitivity analysis (this may take a few minutes)...")
sensitivity_results = prior_sensitivity_analysis(
    model_factory=make_baseline,
    data=sales_train,
    param_grid={"beta_sd": [0.5, 1, 5, 10, 20, 50]},
    fit_kwargs={"method": "mapx", "scaler": "minmax"},
    metric_data=sales_test,
    horizon=len(sales_test),
)

display(sensitivity_results)
print("\nLower RMSE / MAE is better.")

In [ ]:
# Visualise sensitivity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(
    sensitivity_results["beta_sd"], sensitivity_results["rmse"], "o-", color="C0"
)
axes[0].set_xlabel("beta_sd (prior std for Fourier coefficients)")
axes[0].set_ylabel("Test RMSE")
axes[0].set_title("Prior Sensitivity: RMSE")
axes[0].set_xscale("log")
axes[0].grid(True, alpha=0.3)

axes[1].plot(
    sensitivity_results["beta_sd"], sensitivity_results["mape"], "o-", color="C3"
)
axes[1].set_xlabel("beta_sd (prior std for Fourier coefficients)")
axes[1].set_ylabel("Test MAPE (%)")
axes[1].set_title("Prior Sensitivity: MAPE")
axes[1].set_xscale("log")
axes[1].grid(True, alpha=0.3)

plt.suptitle("Baseline Model — Prior Sensitivity Analysis", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("If the curve is flat, the results are robust to the prior choice.")
print("If RMSE changes dramatically, results are prior-sensitive — report this.")

---

## 9. Prior-to-Posterior Visualisation

One of the most intuitive ways to communicate Bayesian results: show how the
data **updates** the prior into the posterior.

For the temperature model, the Fourier coefficients start from a
Normal(0, 10) prior and converge to tight posteriors.

For the transfer model, the priors are the *temperature posteriors* —
already informative — and the short sales data further refines them.

In [ ]:
# Temperature model: prior = Normal(0, 10) for Fourier betas
temp_prior_params = {
    "ft_0 - intercept": {"dist": "normal", "mu": 0, "sigma": 5},
}

# Check which variable names exist in the trace
print("Temperature model variables:")
print(list(temp_model.trace.posterior.data_vars))

In [ ]:
# Build prior_params dict for the Fourier beta coefficients
# Default prior for Fourier betas is Normal(0, beta_sd) where beta_sd defaults to 10
temp_var_names = [v for v in temp_model.trace.posterior.data_vars if "beta" in v]
temp_prior_params = {
    v: {"dist": "normal", "mu": 0, "sigma": 10} for v in temp_var_names
}

# Also include the intercept
intercept_vars = [v for v in temp_model.trace.posterior.data_vars if "intercept" in v]
for v in intercept_vars:
    temp_prior_params[v] = {"dist": "normal", "mu": 0, "sigma": 5}

print(f"Plotting prior→posterior for {len(temp_prior_params)} parameters...")
fig = plot_prior_posterior(temp_model.trace, temp_prior_params)
fig.suptitle("Temperature Model: Prior → Posterior Updating", fontsize=14, y=1.02)
plt.show()

In [ ]:
# Transfer model: prior = Normal(temp_posterior_mean, temp_posterior_std)
# These are the transferred posteriors from the temperature model
beta_key = "fs_0 - beta(p=365.25,n=6)"
beta_posterior = temp_model.trace["posterior"][beta_key]
beta_mean = beta_posterior.mean(dim=["chain", "draw"]).values
beta_std = beta_posterior.std(dim=["chain", "draw"]).values

# Build prior params for the transfer model's Fourier betas
# The transfer model's yearly betas have priors = temperature posteriors
transfer_beta_vars = [
    v for v in transfer_model.trace.posterior.data_vars if "beta" in v and "365" in v
]
transfer_prior_params = {}

if len(transfer_beta_vars) > 0:
    beta_var = transfer_beta_vars[0]
    # The beta is a vector; we need individual entries
    n_coeffs = transfer_model.trace.posterior[beta_var].shape[-1]
    # We'll plot the aggregate distribution
    transfer_prior_params[beta_var] = {
        "dist": "normal",
        "mu": float(beta_mean.mean()),
        "sigma": float(beta_std.mean()),
    }

# Also add weekly betas (default priors)
weekly_beta_vars = [
    v for v in transfer_model.trace.posterior.data_vars if "beta" in v and "7" in v
]
for v in weekly_beta_vars:
    transfer_prior_params[v] = {"dist": "normal", "mu": 0, "sigma": 10}

# Add intercept
intercept_vars = [
    v for v in transfer_model.trace.posterior.data_vars if "intercept" in v
]
for v in intercept_vars:
    transfer_prior_params[v] = {"dist": "normal", "mu": 0, "sigma": 5}

print(f"Plotting prior→posterior for {len(transfer_prior_params)} parameters...")
print("Variables:", list(transfer_prior_params.keys()))

if len(transfer_prior_params) > 0:
    fig = plot_prior_posterior(transfer_model.trace, transfer_prior_params)
    fig.suptitle("Transfer Model: Prior → Posterior Updating", fontsize=14, y=1.02)
    plt.show()
else:
    print("No matching variables found for prior-posterior plot.")

---

## 10. Prediction with Uncertainty

A key advantage of Bayesian models: we get a **full predictive distribution**,
not just a point estimate. This lets us quantify forecast uncertainty via
credible intervals.

In [ ]:
# Generate predictions
baseline_pred = baseline_model.predict(horizon=len(sales_test), freq="D")
transfer_pred = transfer_model.predict(horizon=len(sales_test), freq="D")

# Filter to the relevant date range
transfer_pred = transfer_pred[
    (transfer_pred["ds"] >= sales_train["ds"].min())
    & (transfer_pred["ds"] <= sales_test["ds"].max())
]

print(f"Baseline predictions: {baseline_pred.shape[0]} rows")
print(f"Transfer predictions: {transfer_pred.shape[0]} rows")

In [ ]:
# Side-by-side predictions with uncertainty bands
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

for ax, (name, pred, color) in zip(
    axes,
    [
        ("Baseline (no transfer)", baseline_pred, "C3"),
        ("Transfer Learning", transfer_pred, "C2"),
    ],
):
    # Training data
    ax.plot(sales_train["ds"], sales_train["y"], "C0o", ms=2, alpha=0.4, label="Train")
    # Test data
    ax.plot(sales_test["ds"], sales_test["y"], "C1o", ms=2, alpha=0.4, label="Test")
    # Prediction
    ax.plot(pred["ds"], pred["yhat_0"], f"{color}-", lw=1.5, label=f"{name}")

    # Uncertainty bands (if available from MCMC)
    if "yhat_0_lower" in pred.columns and "yhat_0_upper" in pred.columns:
        ax.fill_between(
            pred["ds"],
            pred["yhat_0_lower"],
            pred["yhat_0_upper"],
            color=color,
            alpha=0.15,
            label="95% credible interval",
        )

    ax.axvline(train_test_date, color="gray", ls="--", lw=2)
    ax.set_title(name)
    ax.set_ylabel("Rides")
    ax.legend(loc="upper right")
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel("Date")
plt.suptitle("Predictions with Uncertainty", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

---

## 11. Quantitative Comparison

Finally, we compute forecast metrics on the held-out test set.

In [ ]:
baseline_metrics = metrics(sales_test, baseline_pred, pool_type="complete")
transfer_metrics = metrics(sales_test, transfer_pred, pool_type="complete")

comparison_df = pd.DataFrame(
    {
        "Baseline (no transfer)": baseline_metrics.iloc[0],
        "Transfer Learning": transfer_metrics.iloc[0],
    },
).T

print("Test Set Metrics")
print("=" * 60)
display(comparison_df.round(2))

# Improvement
for col in ["rmse", "mae", "mape"]:
    imp = (1 - transfer_metrics[col].values[0] / baseline_metrics[col].values[0]) * 100
    print(f"  {col.upper()} improvement: {imp:.1f}%")

---

## 12. Component Plots

In [ ]:
print("Baseline model — component decomposition:")
baseline_model.plot(baseline_pred, y_true=sales_df)

In [ ]:
print("Transfer model — component decomposition:")
transfer_model.plot(transfer_pred, y_true=sales_df)

---

## Summary: Bayesian Workflow Checklist

| Step | Section | Status |
|---|---|---|
| Prior predictive checks | §2 | ✅ Verified priors produce plausible data |
| Convergence diagnostics (R-hat, ESS, BFMI) | §3 | ✅ All models converged |
| Posterior predictive checks | §4 | ✅ Fitted models reproduce observed data |
| Full posterior summaries (HDI, credible intervals) | §6 | ✅ Reported for all parameters |
| Model comparison (LOO / WAIC) | §7 | ✅ Compared baseline vs transfer |
| Prior sensitivity analysis | §8 | ✅ Tested beta_sd ∈ {0.5, 1, 5, 10, 20, 50} |
| Prior-to-posterior visualisation | §9 | ✅ Showed how data updates beliefs |
| Prediction with uncertainty | §10 | ✅ Credible intervals on forecasts |

### Key Takeaways

1. **Always run prior predictive checks** before fitting — they catch impossible priors early.
2. **Convergence diagnostics are non-negotiable** for MCMC. Check R-hat < 1.01 and ESS > 400.
3. **Posterior predictive checks** reveal model misspecification that point metrics miss.
4. **Report full posteriors**, not just point estimates — this is the main advantage of Bayesian methods.
5. **Test prior sensitivity** — especially for short series where priors have outsized influence.
6. **Transfer learning** provides principled, informative priors that make the model robust even with limited data.

### References

- Kruschke, J. K. (2021). Bayesian Analysis Reporting Guidelines. *Nature Human Behaviour*, 5, 1282–1291.
- Vehtari, A., et al. (2017). Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC. *Statistics and Computing*, 27, 1413–1432.
- Gabry, J., et al. (2019). Visualization in Bayesian workflow. *JRSS-A*, 182, 389–402.